In [ ]:
import json
import os
import re
import traceback

import math
import random

import pyzipcode
import hashlib

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown
from IPython.display import HTML
from tqdm import tqdm

import sqlite3
import pandas as pd
import nltk
import scipy
import scipy.stats
import numpy as np

import statsmodels.stats.multitest as multitest
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression,LogisticRegression
import sklearn.preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_validate, LeaveOneOut, KFold
from sklearn.metrics import r2_score

import itertools
import collections
import functools
import collections


In [ ]:
chicago_zips = ['60611', '60614', '60302', '60610', '60607', '60657', '60605',
                '60616', '60654', '60603', '60602', '60631', '60601', '60606',
                '60661', '60660', '60615', '60647', '60618', '60622', '60640',
                '60625', '60642', '60613', '60641', '60646', '60608', '60630',
                '60629', '60623', '60659', '60612', '60609', '60632', '60638',
                '60644', '60643', '60546']


In [ ]:
def flatten_json(v):
    if type(v) == str or type(v) == int or type(v) == float or type(v) == bool or v == None:
        return v
    if type(v) == list:
        new_dict = {}
        for idx, item in enumerate(v):
            flattened = flatten_json(item)
            if type(flattened) == dict:
                for ko,vo in flattened.items():
                    new_dict["%d.%s" % (idx,ko)] = vo
            else:
                new_dict["%d" % (idx)] = flattened
        return new_dict
    elif type(v) == dict:
        new_dict = {}
        for key,value in v.items():
            value = flatten_json(value)
            if type(value) == dict:
                for ko,vo in value.items():
                    new_dict["%s.%s" % (key,ko)] = vo
            else:
                new_dict[key] = value
        return new_dict
    else:
        raise Exception(v,type(v))
            

def get_chicago_businesses():
    for zipcode in chicago_zips:
        with open(f"../data/eyg_data/businesses_by_zipcode/{zipcode}.json") as f:
            businesses = json.load(f)
        for business in businesses:
            if "special_hours" in business:
                del business["special_hours"]
            if "hours" in business:
                del business["hours"]
                
            try:
                yield flatten_json(business)
            except:
                display(business)
                raise

In [ ]:
with open(f"../data/eyg_data/businesses_by_zipcode/{chicago_zips[0]}.json") as f:
    businessid_map = json.load(f)

In [ ]:
businessid_map

In [ ]:
businesses = pd.DataFrame.from_records(get_chicago_businesses())
businesses

In [ ]:
def get_chicago_reviews():
    for zipcode in chicago_zips:
        with open(f"../data/eyg_data/recommended_reviews/{zipcode}.json") as f:
            business_to_reviews = json.load(f)
        for businessID,reviews in business_to_reviews.items():
            for review in reviews:
                review["businessID"] = businessID
                review["flagged"] = "N"
                yield review
        
        with open(f"../data/eyg_data/not_recommended_reviews/{zipcode}.json") as f:
            business_to_reviews = json.load(f)
        for businessID,reviews in business_to_reviews.items():
            for review in reviews:
                review["businessID"] = businessID
                review["flagged"] = "Y"
                yield review

In [ ]:
reviews = pd.DataFrame.from_records(get_chicago_reviews())

In [ ]:
reviews.businessID.unique().size

In [ ]:
businesses.groupby("rating").size()

In [ ]:
businesses

In [ ]:
businesses[businesses.id.isin(reviews.businessID.unique())].groupby("rating").size()

In [ ]:
reviews.to_pickle("../data/pickles/chicago_reviews.pkl")
businesses.to_pickle("../data/pickles/chicago_businesses.pkl")